# FENE beadspring with variable kuhn length experiment data analysis

In [3]:
import os

import polyflexmd.data_analysis.data.read as read
import polyflexmd.data_analysis.transform.transform as transform
import polyflexmd.data_analysis.plot.plot_system as plot_system
import polyflexmd.experiment_runner.config as config
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from pandarallel import pandarallel


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
pandarallel.initialize(
    nb_workers=24,
    progress_bar=False,
    use_memory_fs=False
)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Read and process data

In [6]:
PATH_EXPERIMENT = "/scratch/ws/0/s4610340-bt-eea1-md-workspace/data/experiment_results/FENE-beadspring-free-angle/3-FENE-beadspring-free-angle/a62b82b9"

In [7]:
PATH_EXPERIMENT = pathlib.Path(PATH_EXPERIMENT)
PATH_SYSTEM_DEF = PATH_EXPERIMENT / "data/initial_system.data"
PATH_EC = PATH_EXPERIMENT / "4-FENE-beadspring-vary-l_K.toml"

### Read experiment config

In [ ]:
config.read_experiment_config(PATH_EC)
config

In [ ]:
kappas: list[float] = config.SimulationConfig.variables["kappa"]
kappas

### Read initial system

In [ ]:
system = read.read_lammps_system_data(PATH_SYSTEM_DEF)
system.atoms

In [ ]:
system.bonds

In [ ]:
system.angles

In [ ]:
system.box

### Read and process trajectories

In [ ]:
def read_and_process_kappas_df(experiment_path: pathlib.Path, kappas: list[float]):
    dfs = []
    for i in range(1, len(kappas) + 1):
        paths_trajectories = [
            experiment_path / "data" / f"polymer_relax-{i}.out",
            experiment_path / "data" / f"polymer-{i}.out"
        ]
        df_trajectory_unfolded = transform.unfold_coordinates_df(
            trajectory_df=transform.join_raw_trajectory_df_with_system_data(
                raw_trajectory_df=read.read_multiple_raw_trajectory_dfs(paths_trajectories),
                system_data=system
            ),
            system_data=system
        )
        df_trajectory_unfolded["kappa"] = kappas[i]
        dfs.append(df_trajectory_unfolded)
    return pd.concat(dfs)

df_trajectory_unfolded_kappas = read_and_process_kappas_df(PATH_EXPERIMENT, kappas)
df_trajectory_unfolded_kappas

## Calculate and plot observables

### End-to-End distance $\vec{R}$

In [15]:
PATH_DATA_PROCESSED = PATH_EXPERIMENT / "data" / "processed"
PATH_DATA_PROCESSED.mkdir(exist_ok=True)

PATH_ETE = PATH_EXPERIMENT / "data" / "processed" / "ete.csv"

if PATH_ETE.exists():
    df_ete = pd.read_csv(PATH_ETE, index_col=["molecule-ID", "t"])
else:
    df_ete = df_trajectory_unfolded_kappas.groupby("kappa").apply(transform.calc_end_to_end_df)
    df_ete.to_csv(PATH_ETE, index=True)
df_ete

R_x       R_y       R_z          R
molecule-ID t                                                 
1           0          9.844740 -2.701840  7.548270  12.696268
            100000    10.071300 -3.788600  2.352410  11.014464
            200000    16.600600 -9.010100  4.066660  19.320961
            300000    15.349100 -5.848040  2.344690  16.591926
            400000    16.740700 -3.157660  1.502620  17.102038
...                         ...       ...       ...        ...
120         29960000   3.221090 -0.088719  1.941870   3.762200
            29970000   1.724150 -4.078280  0.471077   4.452749
            29980000   0.270615 -2.586720 -2.036100   3.303037
            29990000   1.273000 -3.735650  1.003840   4.072260
            30000000   3.228840 -1.049590 -0.592467   3.446457

[198120 rows x 4 columns]